# Structural Unit

房室与HH模型是基于离子与离子通道构建的中层模块，负责模拟神经元的空间结构与局部功能整合。其核心包括 ``SingleCompartment``、``MultiCompartment`` 和 ``HHTypeNeuron``，三者共同构成从局部膜区域到完整神经元的结构建模体系。


## 一、SingleCompartment

### 电生理学机制
真实神经元的形态复杂，但在许多场景下，可将神经元的局部区域视为单房室，即一个均匀的、无空间差异的膜区域，忽略其内部的电流空间衰减。

这种简化源于神经科学中的电缆理论：当研究区域的长度远小于电信号的空间常数时，该区域的膜电位可近似为均匀分布，无需考虑空间差异。

单房室的核心是膜电位的时间动态，其物理模型可简化为RC电路：
- **膜电容（Cₘ）**：由隔室的表面积和膜电容密度决定，描述膜储存电荷的能力。
- **膜电阻（Rₘ）**：由膜上离子通道的总电导决定，描述离子流动的阻力。
- **电流平衡**：膜电位变化由跨膜电流决定，遵循方程：
  $$ C_m \frac{dV}{dt} = I_{\text{input}} - I_{\text{ionic}} $$
  其中，$ I_{\text{input}} $ 是外部输入电流，$ I_{\text{ionic}} $ 是离子通道产生的总电流。

### 实现
```python
import braincell
import brainunit as u
import brainstate

# 定义单房室的物理参数
membrane_area = 20000 * u.um**2  # 膜面积
membrane_area = membrane_area.in_unit(u.cm**2)  # 转换为平方厘米
Cm = 1 * u.uF / u.cm**2 * membrane_area  # 膜电容
V_rest = -65. * u.mV  # 静息膜电位
V_th = -20. * u.mV  # 动作电位发放阈值

# 创建单房室模型
single_comp = braincell.SingleCompartment(
    size=1,  # 模拟单个神经元
    C=Cm,  # 膜电容
    V_initializer=brainstate.init.Constant(V_rest),  # 初始膜电位
    V_th=V_th,  # 发放阈值
    solver='ind_exp_euler'  # 数值积分方法
)

# 为单房室添加漏电流通道
single_comp.IL = braincell.channel.IL(
    size=1,
    E=-70. * u.mV,  # 漏电流反转电位
    g_max=0.005 * u.mS / u.cm**2 * membrane_area  # 总漏电导
)

```


## 二、MultiCompartment

### 电生理学机制
当神经元区域的长度接近或超过空间常数时，电信号会随传播距离衰减，单房室模型不再适用。多房室模型通过将神经元分割为多个串联或并联的单房室，模拟真实神经元的空间形态，准确捕捉信号的空间传播特性。

多房室模型基于电缆方程，核心是房室间的轴向电流与膜电流的耦合：
- **轴向电阻（Rₐ）**：描述电流在房室之间流动的阻力，其与隔室长度成正比，与横截面积成反比。
- **房室耦合**：相邻房室的膜电位差驱动轴向电流，遵循
  $$ I_{\text{axial}} = \frac{V_j - V_i}{R_a} $$
  其中，$ V_i 、 V_j  $为相邻房室的膜电位。
- **整体动态**：每个房室的膜电位变化同时受自身离子电流和相邻房室的轴向电流影响，形成微分方程组，准确模拟信号的空间传播与衰减。

### 实现
```python
import braincell
import brainunit as u
import brainstate

# 创建三个单房室（分别代表胞体、树突、轴突）
soma = braincell.SingleCompartment(
    size=1,
    C=1.0 * u.pF,  # 胞体膜电容
    V_initializer=brainstate.init.Constant(-65. * u.mV)
)

dendrite = braincell.SingleCompartment(
    size=1,
    C=0.5 * u.pF,  # 树突膜电容
    V_initializer=brainstate.init.Constant(-65. * u.mV)
)

axon = braincell.SingleCompartment(
    size=1,
    C=0.7 * u.pF,  # 轴突膜电容
    V_initializer=brainstate.init.Constant(-65. * u.mV)
)

# 创建多房室模型并连接各房室
multi_comp = braincell.MultiCompartment(
    compartments=[soma, dendrite, axon],
    solver='euler'  # 数值积分方法
)

# 设置房室间的轴向电阻
multi_comp.set_axial_resistance(
    resistance_matrix=[
        [0, 100, 0],    # 胞体与树突间轴向电阻
        [100, 0, 200],  # 树突与轴突间轴向电阻
        [0, 200, 0]     # 轴突与自身无连接
    ] * u.MOhm
)

# 为每个房室添加漏电流通道
for comp in [soma, dendrite, axon]:
    comp.IL = braincell.channel.IL(
        size=1,
        E=-70. * u.mV,
        g_max=0.05 * u.nS  # 漏电导
    )

```

## 三、HHTypeNeuron

### 电生理学机制
Hodgkin-Huxley 模型是描述动作电位产生机制的经典模型，基于鱿鱼巨轴突的实验数据建立，揭示了Na⁺ 内流触发去极化与K⁺ 外流导致复极化的核心原理。``HHTypeNeuron`` 是对这一模型的结构化封装，属于预设离子通道组合的单房室模型，专为模拟动作电位的产生与传导设计。

``HHTypeNeuron`` 的核心是离子电流的协同作用，基于 ``IonChannel`` 中的 Na⁺ 通道、K⁺ 通道和漏通道：
- **Na⁺ 电流（Iₙₐ）**：由快速激活门`m`和缓慢失活门`h`控制，负责动作电位的上升相。
- **K⁺ 电流（Iₖ）**：由延迟激活门`n`控制，负责动作电位的下降相和超极化。
- **漏电流（Iₗₑₐₖ）**：恒定的非选择性电流，维持静息电位。
- **膜电位动态**：总电流驱动膜电位变化，方程为：
  $$ C_m \frac{dV}{dt} = I_{\text{input}} - (I_{Na} + I_K + I_{leak}) $$

### 实现
```python
import braincell
import brainunit as u
import brainstate

# 创建HH型神经元,基于鱿鱼巨轴突模型
hh_neuron = braincell.HHTypeNeuron(
    size=1,
    C=1.0 * u.uF / u.cm**2,  # 膜电容密度
    area=5000 * u.um**2,     # 膜表面积
    solver='ind_exp_euler'   # 数值积分方法
)

# 设置神经元的初始状态
hh_neuron.V_initializer = brainstate.init.Constant(-65. * u.mV)  # 初始膜电位
hh_neuron.m_initializer = brainstate.init.Constant(0.05)         # Na⁺通道激活门初始值
hh_neuron.h_initializer = brainstate.init.Constant(0.6)          # Na⁺通道失活门初始值
hh_neuron.n_initializer = brainstate.init.Constant(0.32)         # K⁺通道激活门初始值

# 配置Na⁺通道参数
hh_neuron.INa.g_max = 120. * u.mS / u.cm**2  # 最大电导密度
hh_neuron.INa.E = 50. * u.mV                 # 反转电位

# 配置K⁺通道参数
hh_neuron.IK.g_max = 36. * u.mS / u.cm**2    # 最大电导密度
hh_neuron.IK.E = -77. * u.mV                 # 反转电位

# 配置漏电流参数
hh_neuron.IL.g_max = 0.3 * u.mS / u.cm**2    # 最大电导密度
hh_neuron.IL.E = -54.4 * u.mV                # 反转电位

```

## 四、关系

### 层级依赖
- ``HHTypeNeuron`` 是特殊的 ``SingleCompartment``。``HHTypeNeuron`` 本质是预设了Na⁺ 通道 + K⁺ 通道 + 漏通道组合的单房室，专注于动作电位模拟。
- ``MultiCompartment`` 由 ``SingleCompartment`` 组成。多房室模型通过轴向电阻连接多个单房室，形成复杂空间结构。


### 功能互补
- ``SingleCompartment``：用于简化场景，快速验证离子通道组合的电响应。
- ``MultiCompartment``：用于研究空间因素，如树突整合、轴突传导对神经元功能的影响。
- ``HHTypeNeuron``：用于精准模拟动作电位，复现经典神经电生理现象。

通过这种层级结构，房室与HH模型实现了从局部膜区域到完整神经元的灵活建模，兼顾简化性与生物真实性。